# Algoritmus tervezési  módszerek, programozási paradigmák I.

## (Algorithmic design techniques, programming paradigms - part I.)

## Fájlműveletek, fájlbeolvasás (File handling)

In [1]:
filepath = "./data/P0022_names.txt"

# A felkiáltójellel Jupyterben az operációs rendszer parancsait tudom meghívni, 
# pl. Linux alatt a `head` parancsot",

!head -n 5 $filepath

MARY
PATRICIA
LINDA
BARBARA
ELIZABETH


In [2]:
names = []

f = open(filepath, "r")
for line in f:
    names.append(line)

f.close()

In [3]:
names[:5]

['MARY\n', 'PATRICIA\n', 'LINDA\n', 'BARBARA\n', 'ELIZABETH\n']

In [4]:
names = []

f = open(filepath, "r")
for line in f:
    names.append(line.rstrip())

f.close()

In [5]:
names[:5]

['MARY', 'PATRICIA', 'LINDA', 'BARBARA', 'ELIZABETH']

In [6]:
f = open(filepath, "r")
names = f.read().splitlines()

f.close()

In [7]:
names[:5]

['MARY', 'PATRICIA', 'LINDA', 'BARBARA', 'ELIZABETH']

In [8]:
with open(filepath, "r") as f:
    names = f.read().splitlines()

In [9]:
names[:5]

['MARY', 'PATRICIA', 'LINDA', 'BARBARA', 'ELIZABETH']

A Fibonacci-sorozattal már találkoztunk korábban, most újra megnézzük, hogy hogyan lehet kiszámolni a sorozat $n$-edik tagját.

$$
f_n = \left\{\begin{array}{ll} 1 & n = 0 \text{ vagy } n = 1 \\ f_{n-1} + f_{n-2} & n\geq 2 \end{array}\right.
$$

## Rekurzió


Rekurzív függvény esetén mindig kell egy alapeset (base-case), ami biztosítja, hogy a rekurzív függvényhívások sorozata meg fog állni. A Python nyelv lehetővé tesz a rekurzív függvényhívást, de tény, hogy a konstrukció valamelyest idegen a nyelv filozófiájától. Vannak nyelvek, ahol a rekurzió sokkal természetesebb. Az imperatív nyelvekben inkább a direkt iteráció a természetesebb, mint a rekurzió.

In [10]:
def fibonacci(n):
    if n == 0 or n == 1:
        return 1
    
    return fibonacci(n - 1) + fibonacci(n - 2)


fibonacci(37)

39088169

Ugyanazt a függvényértéket sokszor újra és újra kiszámoljuk

```
f(5) = f(4) + f(3)
     = (f(3) + f(2)) + (f(2) + f(1))
     = [(f(2) + f(1)) + (f(1) + f(0))] + [(f(1) + f(0)) + 1]
     = [(f(1) + f(0) + 1 + (1 + 1)] + [1 + 1 + 1])
     = [1 + 1 + 3] + 3
     = 8
```

In [11]:
nr_calls = 0

def fibonacci(n):
    global nr_calls
    nr_calls += 1
    if n == 0 or n == 1:
        return 1

    return fibonacci(n - 1) + fibonacci(n - 2)


_ = fibonacci(5)
print(nr_calls)

nr_calls = 0
_ = fibonacci(37)
print(nr_calls)

15
78176337


## Tároljuk el a már kiszámolt értékeket! (Memoization)

In [12]:
def fibonacci(n):
    cache = {}
    
    def fib(k):
        if k == 0 or k == 1:
            return 1
        
        if k in cache:
            return cache[k]
        
        result = fib(k - 1) + fib(k - 2)
        cache[k] = result
        return result

    return fib(n)

In [13]:
print(fibonacci(5))
print(fibonacci(37))

8
39088169


In [14]:
fibonacci(1000)

70330367711422815821835254877183549770181269836358732742604905087154537118196933579742249494562611733487750449241765991088186363265450223647106012053374121273867339111198139373125598767690091902245245323403501

In [15]:
fibonacci(10000)

RecursionError: maximum recursion depth exceeded in comparison

Most minden már korábban kiszámolt értéket eltárolunk, de ettől még továbbra is rekurzió amit csinálunk, annak minden hátrányával együtt. Adott $n$-re $f(n)$ értékét úgy számoljuk ki, hogy $n$ értékét csökkentve egyre kisebb értékekre próbáljuk kiszámolni $f(n)$ értékét, azaz felülről haladunk lefelé, a nagyobb $n$-ektől a kisebbek felé.

Mi lenne, ha megpróbálnánk a fordított irányban (alulról felfelé haladva) számolni? Lehetséges lenne-e meghatározni $f(0), f(1), f(2), f(3), ...$ értékét ebben a sorrendben, melynek végre eljutunk a kívánt $f(n)$ értékhez?

In [16]:
def fibonacci(n):
    numbers = [1] * (n + 1)
    
    for k in range(2, n+1):
        numbers[k] = numbers[k-1] + numbers[k-2]
        
    return numbers[n] 


print(fibonacci(5))
print(fibonacci(37))

8
39088169


In [17]:
print(fibonacci(10000))

5443837311356528133873426099375038013538918455469596702624771584120858286562234901708305154793896054117382267597802631738435958475111624143917470264295916992558633411790606304808979353147610846625907275936789915067796008830659796664196582493772180038144115884104248099798469648737533718002816376331778192794110136926275097950980071359671802381471066991264421477525447858767456896380800296226513311135992976272667944140010157580004351077746593580536250246170791805922641467900569075232189586814236784959388075642348375438634263963597073375626009896246266874611204173981940487506244370986865431562684718619562014612664223271181504036701882520531484587581719353352982783780035190252923951783668946766191795388471244102846393544948461445077876252952096188759727288922076853739647586954315917243453719361126374392633731300589616724805173798630636811500308839674958710261952463135244749950520419830518716832162328385979462724591977145462821839969578922379891219943177546970521613108109655995063829726125384

A fenti megoldásban $f(n)$-et egyszerűen ki tudtuk számolni egy táblázatban tárolt, már korábban kiszámolt $f(n-1)$ és $f(n-2)$ értékekből. A módszer szerint az eredeti feladat egy részfeladatának eredményeit táblázatban tároljuk, majd felhasználjuk azokat az eredeti feladat megoldásának előállítására. Ezt az algoritmus-sémát, feladatmegoldási-módszert **dinamikus programozás**nak hívjuk.


Végül vegyük észre, hogy a feladat megoldásához nem szükséges eltárolni az összes Fibonacci-számot, hiszem mindig csak a legutolsó kettőre van szükség.

In [18]:
def fibonacci(n):
    a = 1
    b = 1
    for _ in range(1, n+1):
        a, b = b, a + b

    return a


print(fibonacci(5))
print(fibonacci(37))

8
39088169


Sok olyan algoritmikus feladat van, ahol különböző stratégiákkal érdemes próbálkozni. Most láttunk egy példát, hogy ugyanarra a problémára hogy néz ki több különböző megoldási stratégia.

Általában az a legjobb stratégia, amelynek a futásideje gyors és a szükséges tárhely kicsi. Ezeket a fogalmakat majd algoritmuselméleti órákon teszitek rendbe hamarosan.

## 2-SUM

Adott egy $n\leq 10^5$ szám és egy $n$ hosszú $a$ lista, melynek elemeire $-10^{5} \leq a_i \leq 10^5$ teljesül.

Keressünk olyan $1\leq p<q\leq n$ indexpárt, hogy $a[p] = −a[q]$, ha ilyen létezik.


Például:
```
5
5 4 -5 6 8
```

Ötlet: iteráljunk végig az indexpárokon és vizsgáljuk meg minden esetben, hogy teljesíti-e a feltételeket.

Ezt a megoldási módszert, amiben potenciálisan minden lehetséges jelöltet kipróbálunk, **brute force**-alapú (nyers erő) megoldásnak nevezünk. A brute force megoldás mindig működik, a kérdés inkább az, hogy vajon nem tudunk-e ennél jobbat kitalálni.

In [19]:
def solve_2_sum(a):
    for p, elem1 in enumerate(a):
        for q, elem2 in enumerate(a[p+1:], start=p+1):
            if elem1 == -elem2:
                return p, q
            
    return None


xs = [4, 5, -3, -5, 8]
solve_2_sum(xs)

(1, 3)

In [20]:
import random
import time

random.seed(2112)

xs = [random.randint(-10**5, 10**5) for _ in range(50_000)]

t = time.time()
res = solve_2_sum(xs)
print(time.time() - t)

0.002622842788696289


In [21]:
xs = [1] * 50_000

t = time.time()
res = solve_2_sum(xs)
print(res)
print(time.time() - t)

None
35.59650278091431


**HF**: találjunk ki egy sokkal gyorsabb megoldást erre a feladatra, ami tehát egy tetszőleges, $50000$ hosszú listára legfeljebb $0.05$ másodperc alatt lefut.

## Programozási paradigmák

Programozási paradigma alatt programozási stílust, gondolkodási sémát értünk, amit az adott nyelv feature-készlete diktál. A leggyakoribb paradigmák az alábbi csoportokba sorolhatók, de ennél sokkal finomabb felosztás is elképzelhető:

* imperatív
    * procedurális
    * objektum-orientált
    
* deklaratív
    * funkcionális
    * logikai
    * reaktív

Az **imperatív** paradigma fő eleme az utasítás, ahol a gépet utasítjuk, hogy milyen parancsokat hajtson végre. Az ilyen nyelvek főbb jellemzői, alapvető fogalmai: változók, utasítások (statement), ciklusok.

Ezzel szemben áll a **deklaratív** stílus, amikor egy adott feladat megoldásakor olyan kódot írunk, ami a megoldandó feladat egyfajta leírását tartalmazza és nem a konkrét megvalósítás részleteit. Arra fókuszálunk, hogy mit szeretnénk elérni és nem arra, hogy hogyan.

A modern nyelvek általában több paradigmát támogatnak, egyeseket jobban, másokat kevésbé.

A jelenleg legnépszerűbb programozási nyelvek általában az imperatív (procedurális) és az objektum-orientált paradigmát követik (C++, Java, C#), azonban az utóbbi évtizedben erősödik a funkcionális paradigma is. A Python is ezeket a paradigmákat követi, ezeket engedi a felhasználóinak, de nem egyforma mértékben. A Python elsősorban egy imperatív (deklaratív) nyelv, azaz függvényekbe szervezett utasításokat használ, ugyanakkor objektum-orientált is (Pythonban minden objektum, amint ezt már említettük). Emellett, a nyelvtől némileg idegen módon támogat bizonyos elemeket a funkcionális paradigmából is.

A következőkben a Python nyelv eddig nem tárgyalt programozási paradigmáiról lesz szó.

## Objektum-orientált programozás / paradigma (OOP)

Ez a paradigma az ún. objektumok köré szerveződik, amelyek olyan tárolóegységek, melyek egyszerre tartalmaznak adatokat (melyeket mezőknek, vagy attribútumoknak nevezünk), illetve kódot, függvényeket, ami ezen adatok manipulására szolgál. Ezek neve metódus. Metódusokkal elérhető, hogy az objektum adatait magában az objektumban módosítsuk, így az objektum maga egy állapotot tart fenn (pl. egy bankszámla aktuális állapota).

Az objektum-orientált programozásban ilyen objektumok épülnek fel a futó programban, melyek egymással interakcióban vannak. A legtöbb OO-nyelv alapfogalma az osztály (class), ekkor az objektumok ezen osztályok példányai (instance).

Pythonban az OOP nincs előtérben abban az értelemben, hogy lehet Python kódot írni osztályok nélkül is, mindössze függvényekkel operálva. Bizonyos értelemben nem a Python a legmegfelelőbb nyelv, ha igazán meg akarjuk érteni az OOP fő elveit.

Más OOP nyelvekben (pl. Java-ban) sokkal jobban előtérben vannak ezek a fogalmak és nélkülük egyszerűen nem is lehet kódot írni. Ennek ellenére mégis hasznos megismerni az OOP alapjait, mert sok esetben hasznos lehet létező kód megértésében, illetve közepes vagy nagyobb méretű kódbázis fejlesztésénél.

Az első OOP-nyelv a Smalltalk volt (1972), és a fogalmak fejlődése során némileg átalakulva az OOP paradigma az 1990-es és korai 2000-es évek uralkodó paradigmájává vált.

In [22]:
# A legegyszerűbb osztály

class Person:
    pass


p = Person()
print(p)
print(type(p))

<class '__main__.Person'>


In [23]:
# Person egy osztály

class Person:
    pass


# p a Person osztály egy példánya
p = Person()

# a p példányhoz dinamikusan lehet attribútumokat adni
p.name = "Ann"
p.age = 25

print(p)

print(p.name)
print(p.age)

print(p.__dict__)

Ann
25
{'name': 'Ann', 'age': 25}


In [24]:
# Az __init__ metódus inicializálja az osztály, 
# így az attribútúmok nem utólag lesznek hozzáadva az adott példányhoz  

class Person:
    # Minden metódus első paramétere maga az osztálypéldány
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    
p = Person("Ann", 25) 
print(p)
print(p.name)
print(p.age)

Ann
25


In [25]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def introduction(self):
        return f"Hi, my name is {self.name}!"
    
    
p = Person("Ann", 25)

p.introduction()

'Hi, my name is Ann!'

In [26]:
print(p)

In [27]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f"This is {self.name}, {self.age} year(s) old."
        
    def introduction(self):
        return f"Hi, my name is {self.name}!"
    
    
p = Person("Ann", 25)
print(p)

This is Ann, 25 year(s) old.


Van egy `__str__` metódus is a `__repr__` mellett. Az utóbbi általában magunknak szól, az előbbi a végfelhasználónak. Ha osztályt írunk, `__repr__` metódust mindig érdemes implementálni.

OOP-ből ismert fogalmak lehetnek a `public`, `protected` és `private` attribútumok fogalma. Pythonban nincs kulcsszó ezekre, egyszerűen egy nevezéktani konvenció biztosítja, hogy melyik attribútum érhető el kívülről, és melyeket szeretnénk protected-nek vagy privátnak tekinteni.

Más nyelvekből ismerős getter és setter függvényeket lehet ugyan írni, de ne felejtsük, hogy ez itt Python, nincs kikényszerítve, hogy ilyenek írjunk és nem is feltétlenül szokás.

In [28]:
class Person:
    class_variable = 123
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f"This is {self.name}, {self.age} year(s) old."

    def is_pensioner(self):
        return self.age >= 65
    
    
p = Person(name="Ann", age=25)  

print(p.is_pensioner())
print(p.class_variable)
print(Person.class_variable)

False
123
123


Miért használunk osztályokat? Először is, osztályokat használni nem kötelező, de előbb-utóbb találkoztok velük, mások kódjaiban, vagy könyvtárakból való importok esetén. Néhány érv a használatuk mellett:

* Valamilyen módon összetartozó adatokat akarunk eltárolni egységbe zárva (encapsulation)
* Egy állapotot kell tartani, illetve ezt az állapotot kell tudnunk megfelelő módon változtatni
* Az adatot csak meghatározott módon lehessen manipulálni, a felhasználónak nem kell tudnia, hogy mi a belső implementáció (absztakció)
* Hierarchiába szervezhető adattípusok vannak (öröklődés)

In [29]:
class BankAccount:
    def __init__(self, account_id, initial_amount=0):
        self._id = account_id
        self._amount = initial_amount
        
    def __repr__(self):
        return f"This is account {self._id}."
    
    def amount(self):
        return self._amount
        
    def deposit(self, money):
        self._amount += money
        
    def withdraw(self, money):
        if self._amount >= money:
            self._amount -= money
            return money
        
        # Kivétel dobása helyett ilyenkor akár 0 fabatkát is visszadhatnánk, az egyenleget változatlanul hagyva
        raise Exception("Not enough money on your account.")

In [30]:
account = BankAccount(123)

account.deposit(100)
account.deposit(200)
_ = account.withdraw(250)
print(account)
print(account.amount())

This is account 123.
50


Ez a számla nem túl biztonságos, mert az `_amount` mezőt lehet kívülről manipulálni, csak a konvención és az én jóindulatomon múlt, hogy titokban ne adjak pénzt közvetlenül ehhez a mezőhöz.

In [31]:
account = BankAccount(123)
account._amount = 10000

print(account.amount())

10000


Pythonban lehet ennél szigorúbb hozzáférést adni a tényleg privátnak gondolt mezőkhöz, illetve megfelelő settereket és gettereket itt is be lehet állítani, de mi most ennél tovább nem megyünk.

In [32]:
class BankAccount:
    def __init__(self, account_id, initial_amount=0):
        self._id = account_id
        self._amount = initial_amount
        
    def __repr__(self):
        return f"This is account {self._id}."
    
    @property
    def amount(self):
        return self._amount
        
    def deposit(self, money):
        self._amount += money
        
    def withdraw(self, money):
        if self._amount >= money:
            self._amount -= money
            return money
        
        raise Exception("Not enough money on your account.")

In [33]:
account = BankAccount(123)
account.deposit(100)

print(account)
print(account.amount)

This is account 123.
100


In [34]:
account.amount = 1000

AttributeError: can't set attribute

# A verem adatszerkezet (stack)

A verem nem egy Pythonos adatszerkezet, hanem egy általános, nyelvfüggetlen, ún. absztrakt adattípus. A következő jellemzői vannak:
    
* adatokat tudunk benne tárolni
* a következő operációkat támogatja:
    * bele tudunk tenni egy új elemet (push)
    * ki tudjuk venni a legutoljára betett elemet (pop)
    * meg tudjuk nézni a legutoljára betett elemet (top)
    * meg tudjuk érdezni, hogy üres-e a verem (is_empty)
    
úgy, hogy ezek a műveletek gyorsak (lényegében konstans idejűek). Mivel veremben mindig a legutoljára berakott elem van legfelül, ezért az jön ki legelőször, szokás ezt az adatszerkezetet LIFO-nak nevezni (last in, first out).

Mivel a Python adatszerkezetek között nincs verem, ezért csináljunk egyet. Persze van olyan beépített collection, ami ezeket a követelményeket tudja (melyik?), de ezt a mi, saját verem-implementációnkat használó usernek nem kell tudnia (absztrakció).

![](img/stack.png)

In [35]:
class EmptyStack(Exception):
    pass


class Stack:
    def __init__(self):
        pass
        
    def is_empty(self):
        pass
    
    def push(self, item):
        pass
    
    def pop(self):
        pass
        
    def top(self):
        pass

In [36]:
class Stack:
    def __init__(self):
        self._stack = []
        
    def is_empty(self):
        pass

In [37]:
class Stack:
    def __init__(self):
        self._stack = []
        
    def is_empty(self):
        return len(self._stack) == 0

In [38]:
s = Stack()

print(s.is_empty())

True


In [39]:
class Stack:
    def __init__(self):
        self._stack = []
    
    @property
    def is_empty(self):
        return not bool(self._stack)

In [40]:
s = Stack()

print(s.is_empty)

True


In [41]:
class Stack:
    def __init__(self):
        self._stack = []
    
    @property
    def is_empty(self):
        return not bool(self._stack)
    
    def push(self, item):
        pass

In [42]:
class Stack:
    def __init__(self):
        self._stack = []
    
    @property
    def is_empty(self):
        return not bool(self._stack)
    
    def push(self, item):
        self._stack.append(item)

In [43]:
class Stack:
    def __init__(self):
        self._stack = []
    
    @property
    def is_empty(self):
        return not bool(self._stack)
    
    def push(self, item):
        self._stack.append(item)
        
    def top(self):
        if self.is_empty:
            raise EmptyStack("The stack is empty.")
            
        return self._stack[-1]    

In [44]:
s = Stack()

s.push(10)
s.push(20)

print(s.top())

20


In [46]:
s = Stack()

print(s.top())

EmptyStack: The stack is empty.

In [47]:
class Stack:
    def __init__(self):
        self._stack = []
    
    @property
    def is_empty(self):
        return not bool(self._stack)
    
    def push(self, item):
        self._stack.append(item)
        
    def top(self):
        if self.is_empty:
            raise EmptyStack("The stack is empty.")
            
        return self._stack[-1] 
    
    def pop(self):
        if self.is_empty:
            raise EmptyStack("The stack is empty.")
        
        return self._stack.pop()

In [48]:
s = Stack()

s.push(10)
s.push(20)
x = s.pop()
y = s.pop()
print(x)
print(y)
print(s.is_empty)

20
10
True


Mi van, ha szeretnénk különféle értékekkel inicializálni a vermet?

In [49]:
def f(a, b):
    return a + b

In [50]:
def f(*args):
    print(args)
    return sum(args)


f(1, 2, 3, 4)

(1, 2, 3, 4)


10

In [51]:
def f(*args):
    return sum(args)


xs = [1, 2, 3, 4, 5, 6]
f(*xs)

21

In [52]:
class Stack:
    def __init__(self, *args):
        self._stack = list(args)
    
    @property
    def is_empty(self):
        return not bool(self._stack)
    
    def push(self, item):
        self._stack.append(item)
        
    def top(self):
        if self.is_empty:
            raise EmptyStack("The stack is empty.")
            
        return self._stack[-1] 
    
    def pop(self):
        if self.is_empty:
            raise EmptyStack("The stack is empty.")
        
        return self._stack.pop()

In [55]:
s = Stack(10, 20, 100)


print(s.pop())
print(s.pop())
print(s.is_empty)
print(s.pop())
s.push(100)

100
20
False
10


**HF**: Adott egy kifejezés, ami a `(`, `)`, `[` és `]` karakterekből állhat. Állapítsuk meg, hogy a kifejezés helyesen zárójelezett-e vagy sem.

pl. `([()])` helyes zárójelezés, `([(]))` nem az, mert a szögletes zárójelpár tartalmaz pár nélküli nyitó zárójelet.

**HF**: Készítsünk saját halmaz-implementációt! A halmaz elemeit tároljuk egy belső listában, figyelve arra, hogy a lista később sem tartalmazhat duplikátumokat! 
* lehessen üres halmazt készíteni
* legyen `add` metódusa, amivel hozzá lehet adni új elemet
* lehessen értelmes módon kiprintelni a halmazt

```python
class Set:
    def __init__(self):
        self._elems = []
        pass
    
    def __repr__(self):
        pass
    
    def add(self, x):
        pass
```

Ha van időnk, próbáljuk meg a következő feature-öket is hozzáadni:

* lehessen a halmazt különböző elemekkel inicializálni, pl. `s = Set(1, 2, 3, 1, 10, 1)`
* legyen `pop` metódusa
* legyen definiálva egy `union`, egy `intersection` és egy `difference` metódus is